# Gene Expression Clustering by K-Means
## Adam Kim

*This program was completed for CSI 4352 (Introduction to Data Mining).*

In [ ]:
import os        as myOS    # get current directory
import csv       as myCSV   # handle initial file input
DIMENSION_COUNT  = 12       # number of columns
CLUSTER_SIZE     = 50       # initial cluster size

*For a given gene, this function grabs its cluster.*

In [ ]:
def fetchCurrentOwner(member,clusters):
    
    current_index = -1

    j = 0

    while j < len(clusters):
        
        genesInCluster = clusters[j]

        if member in genesInCluster:
            
            current_index = j
            
            break
        
        j+=1
        
    # Assert that gene exists in exactly one cluster.
    
    assert(current_index != -1)
    
    assert(current_index in clusters)

    return current_index

*This function creates 10 centroids, with each centroid having 12 dimensions.*

In [ ]:
def initalize_centroids(filename):

    # Handle reading from file.
    
    handle = open(filename, newline='')
    
    csvHandle = myCSV.reader(handle, delimiter='\t', quotechar='"')

    # centroids stores 10 points in 12-space

    centroids = []
    
    currentObject = []

    # geneToTimeList maps genes -> [t1,...,t12]

    geneToTimeList = {0: []}
    
    geneToTimeList.pop(0, None)

    rowCount = 0

    # Insert genes & associated times into geneToTimeList

    for row in csvHandle:

        for point in row:

            if point:

                if rowCount not in geneToTimeList:
                    geneToTimeList[rowCount] = [float(point)]
                else:
                    geneToTimeList[rowCount].append(float(point))

        rowCount += 1

    # Build aggregating object for calculating means for a centroid.

    dimensionHeap = [0.0] * DIMENSION_COUNT

    # Iterate over geneToTimeList map

    i = 0
    while i < len(geneToTimeList):

        timeList = geneToTimeList[i]

        j = 0
        while j < DIMENSION_COUNT:

            # Aggregate sum for each of 12 dimensions.

            dimensionHeap[j] += timeList[j]
            j += 1

        # For every 50 genes, calculate means for each of 12 dimensions.

        if i > 0 and i % CLUSTER_SIZE == CLUSTER_SIZE - 1:

            meanList = []

            for singleDim in dimensionHeap:

                avg = singleDim / float(CLUSTER_SIZE)

                # To match output with hint given in announcement, 
                # intermediate calculations were rounded to 2 dec places.

                meanList.append(round(avg, 2))

            # push object containing dim-means of 50 12-dimensional objects

            centroids.append(meanList)

            dimensionHeap = [0.0] * DIMENSION_COUNT

        i += 1

    return (centroids, geneToTimeList)

*This function assigns every gene to a cluster whose centroid is closest to that gene.*  

*Proximity calculation uses Manhattan (L1) distance.*  


In [ ]:
def initalize_clusters(myCentroids, geneToTimeList):

    myClusters = {0:[]}
    myClusters.pop(0,None)

    for gene in geneToTimeList:

        timeList = geneToTimeList[gene]

        i = 0

        distToCentroid = []

        while i < len(myCentroids):

            singleCentroid = myCentroids[i]

            ManhattanDist = 0

            j = 0
            while j < len(timeList):

                ManhattanDist += abs( singleCentroid[j] - timeList[j] )
                j+=1
                
            distToCentroid.append(ManhattanDist)

            i+=1

        winner_index = distToCentroid.index(min(distToCentroid))

        if winner_index in myClusters:
            
            myClusters[winner_index].append(gene)
            
        else:
            
            myClusters[winner_index] = [gene]

    return myClusters

*This function recalculates the centroid for each cluster*

*This recalculation takes the average of its member genes and assigns a new centroid for the cluster*


In [ ]:
def recalculate_centroids(myCentroids, geneTimeMap, myClusters):

    i = 0

    while i < len(myCentroids):

        genesInCluster = myClusters[i]

        clusterSize = len(genesInCluster)

        dimArray = [0]*DIMENSION_COUNT

        for gene in genesInCluster:

            timelist = geneTimeMap[gene]

            j = 0

            while j < len(timelist):

                dimArray[j] += timelist[j]

                j+=1

        k = 0

        newDimMeans = []

        while k < len(dimArray):

            singleDim = dimArray[k] / float(clusterSize)

            # To match output with hint given in announcement,
            # intermediate calculations were rounded to 2 dec places.
            newDimMeans.append( round(singleDim,2) )

            k+=1

        myCentroids[i] = newDimMeans

        i+=1

*This function recalculates clusters after centroids are recalculated.*

*Each gene is assigned to cluster whose centroid it is closest to via the Manhattan (L1) distance.*

*Returns FALSE if no gene is assigned to a different cluster.*

*Returns TRUE  if at least one gene is moved to a different cluster.*


In [ ]:
def recalculate_clusters(myCentroids, geneTimeMap, myClusters):

    changeDetected = False

    for gene in geneTimeMap:

        timeList = geneTimeMap[gene]

        i = 0

        distToCentroid = []

        while i < len(myCentroids):

            currCentroid = myCentroids[i]

            assert(len(timeList)==len(currCentroid))

            ManhattanDist = 0

            j = 0
            while j < len(timeList):

                ManhattanDist += abs( currCentroid[j] - timeList[j] )
                j+=1
                
            distToCentroid.append(ManhattanDist)

            i+=1

        winner_index = distToCentroid.index(min(distToCentroid))

        current_index = fetchCurrentOwner(gene,myClusters)

        if winner_index != current_index:
            
            changeDetected = True

            myClusters[current_index].remove(gene)
            
            myClusters[winner_index].append(gene)

    return changeDetected

Driver for this assignment.

The goal of this program is to cluster genes based on 

In [ ]:
def main():
    
    #(1)  For this assignment, initialize centroid means by row number
    
    #(2)  Compute the mean point of the objects in each cluster as a centroids
    
    myCentroids, geneToTimes = initalize_centroids( get_filename() )

    #(3)  Assign each object to the nearest centroid and generate k new clusters
    myClusters = initalize_clusters(myCentroids, geneToTimes)

    #(4)  Repeat (2) and (3), until NO change of the objects in EACH cluster
    
    changeFound = True
    while changeFound:
        recalculate_centroids(myCentroids, geneToTimes, myClusters)
        changeFound = recalculate_clusters(myCentroids,geneToTimes,myClusters)
    
    # Output to text file according to assignment requirements.  
    
    # Output below
    
    finalResults = []

    for index in myClusters:
        finalResults.append([len(myClusters[index]),sorted(myClusters[index])])

    finalResults.sort(key = lambda k: k[0])

    outFile = open(myOS.path.abspath("output-AdamKim.txt"),'w')

    for result in finalResults:
        print('')
        sentence = str(result[0])+' : ' + str(result[1])
        print(sentence.replace('[','{').replace(']','}'))
        outFile.write('\n'+sentence.replace('[','{').replace(']','}') + '\n')
        print('')

    outFile.close()

################################################################################

if __name__ == "__main__":
    main()

_Program Output:_

These following lines ensure that output follows exact specification in assignment instructions:

In an output file, show each cluster at each line starting with the size of the cluster,

for example, "6: {1, 24, 56, 139, 285, 471}".


In [ ]:
2 : {345, 473}

4 : {424, 453, 454, 455}

5 : {491, 496, 497, 498, 499}

8 : {342, 343, 344, 350, 351, 357, 360, 361}

17 : {293, 306, 307, 308, 309, 310, 311, 314, 315, 316, 317, 320, 321, 378, 380,
381, 429}

18 : {394, 407, 409, 410, 417, 418, 420, 421, 422, 423, 428, 438, 439, 440, 447,
451, 452, 474}

19 : {245, 246, 326, 327, 330, 331, 332, 476, 480, 483, 485, 486, 487, 488, 490,
492, 493, 494, 495}

54 : {333, 335, 336, 337, 338, 339, 340, 341, 346, 347, 348, 349, 352, 353, 354,
355, 356, 358, 359, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 391,
392, 411, 412, 414, 415, 416, 419, 457, 458, 459, 461, 463, 464, 465, 466, 467,
468, 469, 470, 471, 472, 477, 489}

69 : {100, 283, 284, 290, 291, 294, 295, 302, 303, 304, 305, 312, 313, 319, 322,
323, 324, 325, 372, 373, 375, 376, 377, 379, 382, 383, 384, 385, 386, 387, 388,
389, 390, 393, 395, 396, 397, 398, 400, 401, 402, 403, 404, 405, 406, 408, 413,
425, 426, 427, 430, 431, 432, 433, 434, 435, 436, 437, 441, 442, 443, 444, 445,
446, 448, 449, 450, 462, 475}

304 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101,
102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197,
198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 247,
248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263,
264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279,
280, 281, 282, 285, 286, 287, 288, 289, 292, 296, 297, 298, 299, 300, 301, 318,
328, 329, 334, 399, 456, 460, 478, 479, 481, 482, 484}

_Program Input:_

In [ ]:
1.00	1.58	1.05	1.15	1.22	0.54	0.73	0.82	0.82	0.90	0.73	0.75
1.00	1.10	0.97	1.00	0.90	0.67	0.81	0.88	0.77	0.71	0.57	0.46
1.00	0.97	1.00	0.85	0.84	0.72	0.66	0.68	0.47	0.61	0.59	0.65
1.00	1.21	1.29	1.08	0.89	0.88	0.66	0.85	0.67	0.58	0.82	0.60
1.00	1.45	1.44	1.12	1.10	1.15	0.79	0.77	0.78	0.71	0.67	0.36
1.00	1.15	1.10	1.00	1.08	0.79	0.98	1.03	0.59	0.57	0.46	0.39
1.00	1.32	1.35	1.13	1.00	0.91	1.22	1.05	0.58	0.57	0.53	0.43
1.00	1.01	1.38	1.21	0.79	0.85	0.78	0.73	0.64	0.58	0.43	0.47
1.00	0.85	1.03	1.00	0.81	0.82	0.73	0.51	0.24	0.54	0.43	0.51
1.00	1.12	0.92	1.01	0.86	0.86	0.70	0.62	0.36	0.37	0.35	0.38
1.00	1.23	1.21	0.95	0.81	0.91	0.87	0.75	0.40	0.54	0.52	0.52
1.00	0.96	1.19	1.08	0.78	0.61	0.53	0.54	0.34	0.61	0.51	0.46
1.00	0.73	0.85	0.74	0.62	0.57	0.47	0.43	0.42	0.53	0.38	0.30
1.00	0.66	0.49	0.59	0.98	0.85	0.43	0.25	0.23	0.62	0.67	0.55
1.00	0.98	0.82	0.88	0.82	0.88	0.48	0.34	0.50	0.93	0.61	0.75
1.00	1.01	0.81	0.78	0.64	0.89	0.46	0.26	0.51	0.71	0.60	0.53
1.00	1.08	1.17	1.03	1.04	1.05	0.77	0.44	0.64	0.74	0.88	0.67
1.00	0.86	0.47	0.52	0.77	0.48	0.42	0.32	0.20	0.32	0.49	0.41
1.00	0.81	0.49	0.75	0.82	0.60	0.45	0.35	0.29	0.46	0.49	0.54
1.00	0.72	0.53	0.66	0.86	0.57	0.64	0.38	0.42	0.47	0.54	0.61
1.00	0.84	0.69	0.69	0.69	0.70	0.53	0.38	0.31	0.64	0.39	0.39
1.00	0.91	0.86	0.73	1.14	0.67	0.65	0.44	0.42	0.75	0.48	0.54
1.00	0.92	0.81	0.81	1.01	0.70	0.57	0.45	0.50	0.76	0.56	0.62
1.00	0.98	0.64	0.75	1.16	0.75	0.55	0.28	0.39	0.74	0.63	0.57
1.00	1.33	0.80	1.13	1.30	1.01	0.90	0.53	0.53	0.89	0.88	0.75
1.00	1.00	0.74	0.95	0.95	0.89	0.73	0.56	0.39	0.80	0.72	0.56
1.00	1.15	0.94	0.86	1.18	1.07	0.65	0.58	0.52	0.77	0.74	0.83
1.00	0.76	0.97	0.79	0.58	0.49	0.60	0.37	0.42	0.99	0.68	0.54
1.00	0.87	0.98	0.84	0.88	0.58	0.50	0.47	0.41	1.21	0.57	0.65
1.00	0.92	0.80	0.74	0.67	0.59	0.55	0.43	0.48	0.76	0.85	0.68
1.00	0.78	0.82	0.81	0.68	0.51	0.46	0.45	0.37	0.92	0.73	0.76
1.00	0.86	0.67	0.79	0.74	0.62	0.51	0.52	0.52	0.89	0.70	0.77
1.00	0.98	0.73	1.08	0.94	0.72	0.64	0.57	0.57	0.93	0.77	0.78
1.00	1.04	1.01	0.81	0.95	0.63	0.69	0.62	0.49	0.99	0.98	0.83
1.00	0.95	0.95	1.00	0.96	0.68	0.61	0.53	0.47	0.46	0.83	0.43
1.00	1.25	0.98	0.77	0.98	0.56	0.57	0.51	0.40	0.56	0.62	0.74
1.00	0.89	0.80	0.68	0.87	0.66	0.59	0.55	0.50	0.60	0.63	0.70
1.00	1.32	0.99	0.78	0.98	0.60	0.60	0.65	0.47	0.53	0.51	0.56
1.00	1.03	0.83	0.68	1.07	0.85	0.65	0.80	0.65	0.62	0.63	0.66
1.00	0.61	0.69	0.55	0.60	0.49	0.35	0.35	0.24	0.35	0.50	0.57
1.00	0.95	1.33	1.32	0.79	0.54	0.57	0.35	0.36	0.50	0.45	0.48
1.00	0.68	1.06	0.99	0.58	0.42	0.22	0.20	0.28	0.50	0.51	0.53
1.00	0.94	0.95	0.86	0.66	0.60	0.48	0.45	0.48	0.54	0.54	0.68
1.00	0.82	0.85	0.82	0.75	0.55	0.57	0.37	0.50	0.71	0.53	0.65
1.00	0.81	0.88	0.69	0.78	0.58	0.50	0.40	0.53	0.72	0.43	0.65
1.00	1.08	1.28	1.16	0.99	0.68	0.63	0.38	0.40	0.74	0.55	0.53
1.00	0.85	1.10	1.02	0.84	0.68	0.79	0.31	0.36	0.76	0.52	0.58
1.00	0.46	0.82	0.72	0.50	0.42	0.37	0.28	0.20	0.47	0.40	0.54
1.00	0.81	1.22	1.08	0.80	0.68	0.71	0.52	0.44	0.71	0.67	0.79
1.00	0.51	0.73	0.96	0.84	0.77	0.70	0.27	0.43	0.63	0.32	0.40
1.00	0.82	0.95	1.02	0.74	0.85	0.74	0.54	0.65	0.71	0.43	0.55
1.00	1.23	1.03	0.99	1.03	0.87	0.63	0.55	0.42	0.58	0.50	0.71
1.00	0.80	0.66	0.72	0.74	0.68	0.51	0.30	0.36	0.43	0.31	0.34
1.00	0.92	0.76	0.70	0.93	0.86	0.62	0.50	0.65	0.56	0.57	0.53
1.00	0.73	1.31	0.85	0.89	0.60	0.57	0.41	0.41	0.57	0.43	0.40
1.00	0.94	0.99	0.78	1.07	0.60	0.57	0.34	0.36	0.52	0.43	0.38
1.00	1.03	0.91	0.78	0.77	0.68	0.52	0.46	0.50	0.65	0.64	0.60
1.00	1.01	0.93	0.85	0.79	0.67	0.45	0.47	0.44	0.41	0.38	0.43
1.00	0.86	0.85	0.94	0.67	0.78	0.64	0.45	0.53	0.48	0.46	0.40
1.00	0.83	0.65	0.69	0.51	0.63	0.59	0.41	0.32	0.51	0.36	0.40
1.00	0.70	0.76	0.85	0.61	0.72	0.73	0.44	0.47	0.62	0.57	0.52
1.00	1.11	1.05	0.71	0.83	0.72	0.69	0.68	0.60	0.72	0.66	0.57
1.00	0.96	1.20	1.04	0.76	0.89	0.79	0.83	0.63	0.80	0.73	0.57
1.00	1.00	1.06	1.02	0.94	1.04	0.46	0.56	0.84	0.79	0.57	0.62
1.00	1.02	1.06	0.75	0.99	0.65	0.46	0.31	0.26	0.34	0.24	0.27
1.00	0.93	1.20	0.81	0.97	0.78	0.69	0.63	0.41	0.41	0.32	0.33
1.00	1.03	0.61	0.76	1.03	0.72	0.61	0.49	0.35	0.37	0.29	0.28
1.00	1.04	0.87	0.94	0.96	0.80	0.66	0.58	0.43	0.54	0.44	0.51
1.00	0.93	0.67	0.69	0.62	0.75	0.54	0.42	0.37	0.52	0.29	0.25
1.00	0.98	0.88	0.67	0.74	0.64	0.43	0.32	0.22	0.24	0.25	0.46
1.00	0.88	0.77	0.69	0.80	0.56	0.40	0.36	0.22	0.20	0.24	0.46
1.00	1.00	0.96	1.21	1.10	0.95	0.68	0.43	0.46	0.49	0.55	0.58
1.00	1.01	1.03	1.07	1.06	0.85	0.65	0.45	0.43	0.34	0.43	0.50
1.00	0.97	1.22	1.12	0.84	0.94	0.74	0.56	0.51	0.46	0.56	0.46
1.00	1.31	1.45	1.50	1.38	0.93	0.76	0.62	0.59	0.61	0.49	0.56
1.00	1.18	0.73	0.83	0.79	0.61	0.39	0.24	0.17	0.38	0.35	0.38
1.00	0.93	0.77	0.84	0.74	0.66	0.50	0.35	0.31	0.52	0.43	0.44
1.00	0.91	0.92	0.81	0.73	0.51	0.36	0.30	0.23	0.37	0.32	0.36
1.00	1.08	0.84	0.95	0.94	0.82	0.59	0.54	0.24	0.49	0.48	0.63
1.00	0.88	0.86	0.67	0.72	0.83	0.54	0.50	0.34	0.45	0.50	0.47
1.00	1.06	1.00	1.02	1.07	1.16	0.68	0.49	0.40	0.55	0.82	0.91
1.00	1.32	1.27	0.87	1.04	1.05	0.81	0.52	0.43	0.69	0.70	0.69
1.00	1.11	1.03	0.76	0.80	0.60	0.52	0.45	0.32	0.33	0.51	0.61
1.00	0.74	0.95	0.89	0.73	0.36	0.20	0.22	0.19	0.23	0.16	0.19
1.00	0.81	0.93	0.75	1.18	0.54	0.31	0.34	0.23	0.31	0.36	0.36
1.00	1.05	0.96	0.98	0.87	0.54	0.31	0.31	0.41	0.48	0.36	0.36
1.00	1.08	1.47	1.46	1.02	0.58	0.49	0.49	0.63	0.67	0.57	0.62
1.00	1.30	1.27	1.07	1.05	0.64	0.42	0.48	0.61	0.70	0.58	0.57
1.00	1.05	1.27	0.86	1.04	0.61	0.49	0.55	0.52	0.58	0.61	0.56
1.00	1.16	1.20	0.93	0.86	0.49	0.45	0.53	0.45	0.56	0.64	0.64
1.00	1.05	1.13	1.01	0.77	0.43	0.33	0.51	0.49	0.60	0.50	0.69
1.00	0.85	1.07	1.05	0.83	0.59	0.39	0.61	0.67	0.55	0.69	0.69
1.00	1.09	0.92	1.12	1.16	0.53	0.48	0.70	0.59	0.54	0.42	0.41
1.00	1.20	1.21	0.81	0.81	0.69	0.51	0.47	0.56	0.43	0.72	0.75
1.00	1.12	1.04	1.02	0.79	0.77	0.51	0.49	0.69	0.61	0.85	0.99
1.00	0.95	0.78	1.01	0.85	0.88	0.54	0.43	0.57	0.57	0.69	0.74
1.00	0.70	1.56	1.24	0.79	0.74	0.49	0.52	0.42	0.64	0.67	0.65
1.00	0.72	0.84	0.76	0.79	0.87	0.50	0.54	0.45	0.70	0.56	0.78
1.00	0.97	0.83	1.09	1.01	1.10	0.69	0.61	0.69	0.49	0.66	0.75
1.00	1.12	1.64	1.18	0.76	0.80	0.62	0.58	0.47	0.60	0.55	0.49
1.00	2.02	0.98	1.08	1.05	0.46	0.50	0.80	1.50	1.91	1.90	1.86
1.00	1.25	0.81	0.63	1.05	0.45	0.51	0.67	1.41	1.82	1.40	1.59
1.00	1.17	0.62	0.77	0.89	0.58	0.61	0.67	1.03	1.58	1.34	1.56
1.00	0.98	0.57	0.87	0.72	0.45	0.48	0.76	0.69	1.11	1.07	1.21
1.00	0.86	0.77	0.97	0.83	0.51	0.70	0.82	0.86	1.22	1.62	1.68
1.00	0.76	0.71	0.79	0.56	0.50	0.49	0.63	1.13	0.95	1.21	1.28
1.00	1.01	0.65	1.10	0.93	0.44	0.35	0.37	0.79	1.07	1.01	0.87
1.00	1.01	0.85	0.98	0.84	0.25	0.25	0.27	0.51	0.86	0.89	1.18
1.00	0.97	0.77	0.92	0.91	0.52	0.40	0.39	0.94	1.02	1.30	1.47
1.00	0.90	0.59	0.44	0.64	0.33	0.23	0.33	0.58	0.94	0.71	0.72
1.00	0.81	0.63	0.77	0.96	0.45	0.41	0.48	0.65	0.69	0.82	1.01
1.00	0.97	0.54	0.75	0.85	0.47	0.48	0.50	0.60	0.77	0.83	0.83
1.00	0.70	0.52	0.70	1.01	0.46	0.39	0.59	0.59	0.73	0.74	0.69
1.00	0.78	0.63	0.70	0.87	0.49	0.54	0.75	0.82	0.80	0.86	0.96
1.00	1.06	0.61	0.85	0.87	0.56	0.65	0.74	0.76	1.10	0.90	0.85
1.00	1.09	0.92	0.77	1.01	0.72	0.88	0.73	0.80	1.13	1.11	0.98
1.00	0.94	0.62	0.67	0.70	0.72	0.93	0.84	0.92	1.21	1.06	1.21
1.00	0.89	0.72	0.57	0.99	0.70	0.62	0.47	0.62	1.01	1.00	1.20
1.00	0.95	0.77	0.60	1.04	0.84	0.81	0.62	0.74	1.08	1.20	1.23
1.00	0.92	0.61	0.56	0.81	0.73	0.67	0.67	0.58	0.82	1.18	1.12
1.00	0.97	0.68	0.83	0.84	0.96	0.60	0.58	0.70	0.78	0.92	1.07
1.00	0.98	0.62	0.76	0.93	0.71	0.60	0.50	0.87	0.86	1.13	1.10
1.00	0.82	0.30	0.57	0.78	0.68	0.52	0.34	0.43	0.83	0.86	1.06
1.00	0.79	0.36	0.50	0.49	0.68	0.45	0.38	0.36	0.74	0.90	0.71
1.00	0.64	0.39	0.38	0.75	0.75	0.59	0.46	0.42	1.08	1.31	1.35
1.00	0.94	0.54	0.66	0.59	0.68	0.80	0.78	0.60	0.89	1.16	1.13
1.00	0.90	0.88	0.65	1.31	1.04	0.92	0.76	0.67	0.98	1.10	1.07
1.00	0.85	0.76	0.90	0.69	0.23	0.35	0.34	0.28	0.55	0.46	0.51
1.00	1.02	0.94	0.88	0.98	0.39	0.45	0.41	0.36	0.59	0.54	0.57
1.00	0.96	0.73	0.66	0.95	0.34	0.24	0.18	0.36	0.64	0.56	0.56
1.00	1.17	1.03	0.99	0.78	0.30	0.16	0.16	0.19	0.68	0.66	0.64
1.00	0.94	1.15	0.90	0.97	0.67	0.52	0.49	0.50	0.85	0.94	1.01
1.00	0.86	0.99	0.97	0.71	0.62	0.49	0.47	0.46	0.71	0.71	0.87
1.00	1.10	0.82	0.88	0.83	0.53	0.44	0.33	0.47	0.86	0.71	0.73
1.00	1.07	0.80	0.77	1.00	0.55	0.49	0.39	0.41	0.77	0.67	0.77
1.00	1.05	0.78	0.71	0.85	0.65	0.46	0.43	0.44	0.74	0.84	0.81
1.00	0.85	1.00	0.90	0.77	0.60	0.37	0.30	0.52	0.96	0.73	0.72
1.00	1.09	0.89	0.91	0.62	0.49	0.33	0.32	0.45	0.82	0.64	0.67
1.00	1.26	0.89	0.98	0.94	0.58	0.48	0.49	0.57	0.83	0.64	0.70
1.00	1.07	0.76	0.81	0.87	0.49	0.47	0.48	0.57	0.78	0.66	0.62
1.00	1.27	0.98	1.25	0.89	0.34	0.21	0.22	0.30	0.57	0.48	0.51
1.00	0.90	0.84	1.03	0.66	0.60	0.40	0.41	0.45	0.63	0.62	0.67
1.00	1.01	0.88	0.98	0.70	0.42	0.29	0.26	0.32	0.52	0.52	0.74
1.00	1.00	1.18	1.32	1.14	0.88	0.30	0.30	0.42	0.61	0.58	0.77
1.00	1.05	0.98	0.99	1.17	0.86	0.45	0.36	0.38	0.62	0.68	0.91
1.00	1.05	1.14	0.91	1.06	0.64	0.46	0.38	0.38	0.68	0.56	0.60
1.00	1.02	1.26	1.22	1.13	0.75	0.56	0.49	0.49	0.80	0.85	0.71
1.00	0.79	0.73	0.73	0.88	0.72	0.33	0.39	0.33	0.66	0.64	0.68
1.00	1.27	1.16	1.24	0.90	0.91	0.72	0.64	0.69	0.68	0.81	1.22
1.00	0.97	1.02	1.17	0.77	0.94	0.53	0.47	0.58	0.95	0.91	0.80
1.00	0.98	0.81	0.90	0.88	0.68	0.38	0.41	0.58	0.57	0.85	0.98
1.00	1.31	0.92	1.17	1.03	0.88	0.56	0.49	0.62	0.81	1.04	1.10
1.00	0.91	0.61	0.75	0.72	0.82	0.42	0.38	0.39	0.59	0.87	0.81
1.00	0.69	0.64	0.54	0.73	0.50	0.46	0.24	0.26	0.84	0.69	0.63
1.00	1.06	0.84	0.93	0.90	0.69	0.44	0.40	0.37	1.08	0.65	0.76
1.00	1.04	0.92	0.83	0.90	0.74	0.53	0.40	0.47	0.98	0.81	0.90
1.00	0.77	0.56	0.67	0.72	0.64	0.46	0.40	0.34	0.73	0.64	0.64
1.00	0.97	0.67	0.78	0.79	0.59	0.43	0.32	0.37	0.76	0.64	0.52
1.00	0.92	0.90	0.79	0.90	0.56	0.40	0.36	0.47	0.70	0.72	0.60
1.00	0.93	0.61	0.76	0.94	0.42	0.26	0.20	0.31	0.70	0.74	0.76
1.00	0.90	0.68	0.94	0.87	0.68	0.50	0.36	0.41	0.78	0.69	0.81
1.00	0.98	0.58	0.99	1.08	0.59	0.45	0.26	0.44	0.76	0.88	0.86
1.00	1.15	0.83	0.87	1.00	0.88	0.67	0.46	0.38	1.00	0.96	0.96
1.00	0.98	0.85	0.60	0.87	0.56	0.46	0.33	0.26	0.66	0.62	0.87
1.00	1.29	0.89	0.71	0.90	0.58	0.47	0.39	0.26	0.65	0.82	0.83
1.00	0.82	0.87	0.93	0.93	0.69	0.58	0.36	0.29	0.67	0.71	0.94
1.00	1.01	0.85	1.12	0.90	0.76	0.54	0.45	0.32	0.78	0.65	0.80
1.00	0.94	0.80	0.78	0.71	0.60	0.44	0.37	0.33	0.62	0.60	0.57
1.00	0.79	0.68	0.75	0.82	0.43	0.35	0.40	0.31	0.70	0.59	0.72
1.00	0.76	0.69	0.90	0.87	0.52	0.51	0.49	0.35	0.81	0.76	0.84
1.00	0.85	0.71	0.70	0.91	0.47	0.66	0.42	0.34	0.77	0.78	0.83
1.00	1.08	0.88	0.81	1.03	0.64	0.54	0.44	0.39	0.73	0.74	0.97
1.00	1.17	0.75	0.69	0.83	0.68	0.53	0.63	0.49	0.63	0.66	0.96
1.00	1.09	0.73	0.78	1.01	0.59	0.56	0.63	0.60	0.84	0.78	0.86
1.00	1.36	0.99	0.75	1.19	0.53	0.57	0.58	0.46	0.99	0.83	0.93
1.00	0.93	0.62	0.67	0.68	0.58	0.39	0.42	0.46	0.89	0.92	0.96
1.00	0.94	0.59	0.77	0.70	0.54	0.39	0.40	0.46	0.59	0.70	0.94
1.00	0.72	0.59	0.62	0.79	0.51	0.43	0.38	0.40	0.61	0.59	0.71
1.00	0.81	0.53	0.69	0.94	0.69	0.37	0.44	0.41	0.95	0.88	0.83
1.00	0.71	0.86	0.51	0.80	0.49	0.32	0.34	0.29	0.95	0.74	0.65
1.00	0.63	0.74	0.70	1.04	0.58	0.44	0.47	0.49	0.96	0.96	0.81
1.00	0.50	0.71	0.64	0.49	0.55	0.46	0.43	0.37	0.87	0.94	0.98
1.00	0.69	0.71	0.78	0.71	0.57	0.46	0.65	0.44	0.74	0.87	0.99
1.00	0.69	0.71	0.63	0.77	0.75	0.58	0.58	0.42	0.87	0.96	1.23
1.00	1.04	0.84	0.95	0.78	0.58	0.68	0.61	0.61	1.15	1.22	1.40
1.00	0.81	0.81	1.11	1.01	0.50	0.60	0.55	0.55	1.10	1.00	1.11
1.00	0.90	0.88	0.87	1.14	0.62	0.68	0.71	0.66	1.08	1.35	1.40
1.00	1.00	0.93	0.86	0.87	0.65	0.56	0.75	0.70	1.15	1.01	1.22
1.00	1.25	1.10	1.02	0.97	0.54	0.59	0.83	0.73	1.08	1.24	1.32
1.00	0.93	0.93	0.93	0.63	0.42	0.69	0.72	0.55	1.06	1.04	1.08
1.00	0.60	0.79	0.58	0.49	0.36	0.32	0.28	0.34	0.63	0.81	1.03
1.00	0.67	0.82	0.82	0.56	0.43	0.47	0.39	0.47	0.85	0.71	0.74
1.00	0.58	0.75	0.72	0.49	0.51	0.71	0.33	0.42	0.78	0.70	0.89
1.00	0.74	0.85	0.92	0.78	0.36	0.36	0.28	0.28	0.88	0.99	0.91
1.00	0.90	0.91	0.95	0.86	0.62	0.52	0.43	0.38	0.94	1.23	1.13
1.00	0.89	1.04	1.07	1.01	0.68	0.56	0.42	0.38	0.79	0.83	0.97
1.00	0.87	1.23	1.14	0.89	0.69	0.63	0.58	0.49	0.88	0.97	1.25
1.00	0.75	0.98	0.91	0.70	0.43	0.40	0.40	0.44	0.75	0.77	0.78
1.00	0.98	1.32	1.11	1.07	0.57	0.44	0.51	0.44	0.82	0.76	1.03
1.00	0.64	1.49	1.05	0.96	0.68	0.66	0.59	0.79	0.85	1.16	1.30
1.00	1.18	0.58	0.65	0.84	0.49	0.31	0.21	0.17	0.73	1.19	1.41
1.00	0.93	0.56	0.72	0.86	0.63	0.34	0.20	0.27	0.70	1.33	1.50
1.00	1.08	0.82	0.91	1.17	0.66	0.49	0.42	0.51	0.93	1.23	1.61
1.00	0.76	1.07	1.01	0.75	0.60	0.38	0.31	0.28	0.87	1.04	1.37
1.00	1.01	0.79	0.92	0.81	0.54	0.42	0.37	0.32	0.80	1.23	1.07
1.00	0.87	0.72	0.82	0.96	0.56	0.46	0.55	0.38	0.96	0.96	1.17
1.00	1.14	0.77	0.92	1.10	0.61	0.54	0.66	0.50	1.01	1.05	1.16
1.00	0.88	0.82	0.86	0.83	0.56	0.39	0.56	0.49	0.83	0.97	1.25
1.00	0.99	0.62	0.67	0.68	0.61	0.40	0.28	0.27	0.89	0.93	1.15
1.00	0.77	0.58	0.63	0.68	0.58	0.37	0.32	0.37	0.55	0.65	0.72
1.00	0.47	0.26	0.39	0.58	0.17	0.16	0.14	0.19	0.59	0.74	0.90
1.00	0.83	0.45	0.49	0.77	0.52	0.40	0.31	0.37	0.63	0.71	0.88
1.00	0.93	0.50	0.70	0.90	0.55	0.47	0.44	0.42	0.76	0.78	1.03
1.00	0.68	0.61	0.77	0.86	0.62	0.49	0.42	0.54	0.90	0.88	0.93
1.00	0.67	0.54	0.53	0.78	0.51	0.46	0.40	0.42	0.84	0.71	0.78
1.00	0.58	0.34	0.51	0.54	0.41	0.23	0.24	0.24	0.57	0.50	0.59
1.00	0.90	0.71	1.02	1.06	0.77	0.48	0.47	0.53	1.29	0.87	0.94
1.00	1.64	1.02	1.54	1.19	1.13	0.71	0.69	0.62	1.51	1.24	1.76
1.00	1.05	0.71	0.80	0.68	0.70	0.59	0.53	0.59	1.01	0.99	0.96
1.00	1.07	1.72	1.21	0.77	0.87	0.91	0.60	0.94	1.08	0.99	1.06
1.00	0.89	1.00	1.21	0.77	1.21	0.49	0.44	0.63	1.04	0.82	1.29
1.00	0.63	0.89	0.94	0.57	0.83	0.53	0.29	0.46	0.76	0.60	0.74
1.00	1.17	0.76	0.89	1.03	0.75	0.76	0.60	0.55	0.97	0.84	1.23
1.00	1.25	1.09	1.05	1.32	0.88	0.73	0.70	0.63	1.05	1.31	1.58
1.00	0.79	0.55	0.74	0.96	0.56	0.57	0.36	0.52	0.74	0.83	0.87
1.00	1.15	1.41	0.92	1.09	0.74	0.50	0.48	0.77	0.89	0.85	1.00
1.00	1.04	1.02	0.99	1.09	0.63	0.44	0.63	0.77	0.94	0.96	0.97
1.00	1.09	1.09	0.98	1.47	0.75	0.51	0.55	0.35	0.71	1.00	1.15
1.00	0.85	0.84	0.68	1.10	0.41	0.39	0.58	0.39	0.80	0.82	0.85
1.00	0.93	0.77	0.78	0.71	0.71	0.66	0.46	0.62	0.55	0.89	1.22
1.00	0.65	0.97	1.11	0.79	0.62	0.37	0.41	0.66	1.00	1.00	1.10
1.00	0.58	0.88	0.77	0.39	0.41	0.26	0.30	0.29	0.70	0.84	1.03
1.00	1.01	0.96	0.96	1.19	0.55	0.22	0.10	0.18	0.79	0.97	1.07
1.00	1.04	0.87	0.85	0.79	0.35	0.19	0.20	0.28	0.73	0.72	0.79
1.00	1.11	1.02	1.05	0.96	0.53	0.29	0.38	0.47	0.88	0.79	1.02
1.00	1.33	1.01	1.06	1.17	0.70	0.32	0.37	0.37	0.79	1.06	1.17
1.00	0.95	1.24	0.95	1.17	0.44	0.30	0.39	0.51	0.71	0.75	0.86
1.00	0.99	1.02	1.05	1.14	0.59	0.34	0.37	0.46	0.69	0.57	0.84
1.00	0.96	1.40	1.15	0.85	0.50	0.36	0.58	0.52	1.26	1.28	1.28
1.00	1.00	1.26	0.81	0.90	0.57	0.52	0.51	0.52	0.90	0.70	0.84
1.00	0.66	0.67	1.03	0.84	0.49	0.74	0.45	0.48	0.79	0.74	0.59
1.00	0.81	0.60	0.86	0.67	0.43	0.64	0.50	0.48	0.70	0.75	0.81
1.00	1.04	0.85	1.23	0.96	0.46	0.62	0.66	0.47	1.08	0.91	0.91
1.00	0.78	1.34	1.42	0.81	0.70	0.55	0.50	0.48	0.86	0.88	1.07
1.00	0.77	1.25	1.20	0.80	0.72	0.60	0.45	0.36	0.60	0.66	0.85
1.00	2.94	4.08	2.74	2.48	2.68	2.61	2.15	2.32	1.34	1.35	1.23
1.00	3.35	6.95	4.82	3.76	3.70	3.14	2.91	2.02	2.16	1.98	1.60
1.00	0.60	1.30	1.78	1.14	1.19	0.94	0.87	0.56	1.10	0.94	1.05
1.00	0.86	1.63	1.43	0.73	0.86	1.06	0.69	0.53	0.70	0.67	0.67
1.00	0.90	1.67	1.38	0.71	1.22	0.85	0.42	0.50	0.74	0.88	0.73
1.00	0.83	1.31	1.64	0.66	1.24	0.80	0.36	0.45	0.51	0.64	0.61
1.00	0.71	0.98	0.93	0.57	0.69	0.75	0.39	0.30	0.41	0.34	0.30
1.00	0.50	1.63	1.80	0.61	0.96	0.61	0.44	0.26	0.53	0.60	0.57
1.00	0.96	1.51	1.16	0.93	0.93	0.84	0.79	0.54	0.33	0.25	0.21
1.00	1.27	1.72	1.26	1.05	1.03	1.09	1.09	0.94	0.53	0.52	0.43
1.00	1.04	1.63	1.10	0.94	0.86	0.85	0.91	0.85	0.61	0.56	0.46
1.00	1.15	1.62	1.29	0.78	0.99	1.09	0.85	0.63	0.57	0.44	0.41
1.00	1.04	1.91	1.42	1.04	1.08	0.96	0.82	0.76	0.63	0.59	0.56
1.00	1.44	1.82	1.43	1.18	1.16	1.31	1.28	0.82	1.19	0.59	0.57
1.00	1.30	1.78	1.35	1.07	1.25	1.37	1.50	0.79	0.51	0.50	0.43
1.00	0.98	1.47	0.92	0.81	0.72	0.58	0.75	0.84	0.31	0.51	0.37
1.00	1.18	1.61	1.03	1.04	0.88	0.98	0.65	0.73	0.45	0.51	0.42
1.00	0.90	0.80	1.04	1.05	0.78	0.46	0.29	0.17	0.19	0.22	0.23
1.00	0.91	1.14	1.20	0.97	1.10	0.82	0.59	0.44	0.65	0.50	0.42
1.00	1.12	1.15	0.99	0.81	0.71	0.57	0.48	0.28	0.28	0.30	0.37
1.00	0.98	1.17	0.98	0.75	0.81	0.77	0.54	0.37	0.31	0.26	0.25
1.00	1.03	1.01	0.96	1.03	0.95	0.75	0.62	0.49	0.42	0.38	0.41
1.00	1.31	1.06	1.23	1.22	0.84	0.93	0.80	0.58	0.48	0.54	0.61
1.00	1.71	1.84	2.18	1.70	1.14	0.85	0.68	0.59	0.53	0.46	0.44
1.00	0.89	0.89	1.44	1.05	1.49	0.76	0.34	0.23	0.26	0.25	0.30
1.00	1.15	1.61	1.80	2.11	1.30	0.83	0.33	0.20	0.24	0.24	0.24
1.00	1.11	1.56	1.48	1.98	1.32	0.82	0.39	0.27	0.25	0.25	0.27
1.00	0.94	1.04	1.48	1.66	1.24	0.80	0.52	0.33	0.42	0.44	0.53
1.00	0.85	1.04	1.86	3.03	1.56	0.70	0.43	0.13	0.16	0.13	0.12
1.00	0.93	2.16	2.27	0.90	1.04	0.83	0.67	0.72	0.44	0.47	0.47
1.00	1.23	2.58	3.15	1.37	1.50	1.44	1.38	0.67	0.70	0.64	0.59
1.00	1.27	2.42	1.51	1.04	0.63	0.73	1.17	0.62	0.58	0.55	0.39
1.00	1.14	1.94	1.42	1.01	1.04	0.83	0.75	0.41	0.23	0.20	0.17
1.00	1.29	1.30	1.60	1.52	1.38	1.35	1.60	0.82	0.59	0.47	0.37
1.00	1.04	0.65	0.92	1.16	0.84	0.56	0.82	2.27	1.80	1.14	0.96
1.00	0.81	1.07	0.32	0.93	0.99	0.78	1.19	1.15	0.82	0.70	0.17
1.00	1.34	1.01	0.64	1.76	1.64	1.71	1.81	1.53	0.95	0.84	0.70
1.00	0.97	0.66	1.02	2.07	2.11	1.61	1.65	0.73	0.65	0.55	0.58
1.00	1.11	1.11	0.63	1.22	0.77	0.61	0.51	0.53	1.54	2.44	3.18
1.00	1.17	0.93	0.92	1.00	0.66	0.67	0.52	0.58	1.49	2.79	4.44
1.00	1.11	0.77	0.93	0.85	0.81	0.56	0.57	0.72	1.40	1.57	2.20
1.00	1.02	0.67	0.56	0.84	0.50	0.41	0.46	0.58	1.18	1.49	1.40
1.00	1.07	0.94	0.72	0.91	0.62	0.44	0.55	0.58	0.85	1.15	1.37
1.00	0.88	0.83	0.77	0.87	0.62	0.44	0.45	0.46	1.04	0.91	1.51
1.00	1.14	0.64	0.82	1.19	0.68	0.75	0.68	0.87	1.26	1.51	1.80
1.00	1.13	0.93	1.10	1.46	0.63	0.76	0.77	0.65	1.91	2.61	3.02
1.00	1.01	0.87	1.09	1.09	0.67	0.50	0.86	0.67	1.48	2.18	2.56
1.00	0.86	0.94	1.02	1.03	0.70	0.58	0.81	0.59	1.33	1.48	1.38
1.00	1.37	1.25	1.28	1.66	0.65	0.78	0.68	1.69	2.47	3.63	5.03
1.00	1.29	1.25	0.83	1.07	0.60	0.63	0.87	1.14	1.63	2.27	2.41
1.00	1.43	0.76	0.95	1.04	0.82	1.42	0.61	1.23	1.59	2.61	2.60
1.00	0.96	0.65	0.91	1.01	0.99	0.84	0.66	1.05	1.76	1.57	1.98
1.00	1.51	1.07	1.30	0.60	0.24	0.18	0.27	0.61	1.17	1.07	0.97
1.00	0.93	0.92	0.67	0.77	0.30	0.38	0.27	0.43	0.83	0.83	0.90
1.00	0.76	1.14	0.88	0.82	0.38	0.33	0.49	0.54	1.00	1.22	1.28
1.00	0.85	0.77	0.76	0.63	0.31	0.42	0.62	0.73	1.11	0.99	1.17
1.00	1.41	1.35	1.41	1.12	0.80	0.49	0.40	0.62	0.87	1.53	2.46
1.00	1.21	0.86	0.89	1.04	0.65	0.66	0.70	1.20	2.53	2.95	2.48
1.00	1.21	0.80	0.96	1.10	0.74	0.57	0.72	1.08	2.77	3.34	1.41
1.00	1.15	0.50	0.68	0.79	0.64	0.71	0.63	1.45	1.72	2.78	2.36
1.00	0.92	0.78	0.72	0.57	0.40	0.39	0.75	1.53	2.38	2.96	3.46
1.00	1.54	1.57	1.11	1.14	0.71	1.08	0.99	1.19	2.33	4.85	7.36
1.00	1.33	1.65	1.21	1.39	0.66	0.78	0.87	0.86	2.45	4.78	8.83
1.00	1.21	1.59	1.44	1.06	0.83	0.97	0.71	0.73	1.98	2.94	6.49
1.00	1.30	2.21	1.81	1.34	0.96	0.97	0.62	0.84	2.64	4.49	8.08
1.00	1.11	1.18	1.07	1.00	0.77	0.80	0.65	0.92	3.09	5.73	8.84
1.00	1.24	1.45	1.39	1.22	0.91	1.22	0.94	1.04	2.55	4.22	6.98
1.00	0.90	0.68	0.85	1.24	0.64	0.57	0.75	1.06	2.34	2.93	3.44
1.00	1.08	0.81	1.07	1.01	0.77	0.77	1.08	1.19	1.37	2.77	4.50
1.00	1.06	0.99	1.05	1.26	1.05	1.00	1.01	1.41	3.14	4.23	5.23
1.00	1.41	1.46	0.97	1.63	0.97	0.98	0.92	1.55	2.98	4.40	5.84
1.00	1.30	1.46	1.23	0.83	0.99	0.96	0.93	1.36	3.25	5.15	9.73
1.00	0.82	1.01	0.98	0.87	0.93	0.96	1.08	1.67	3.42	5.71	6.62
1.00	1.06	0.77	0.94	0.79	0.75	0.66	0.55	0.44	1.47	2.03	4.32
1.00	0.84	0.69	0.93	1.09	0.90	0.90	1.01	0.73	1.29	2.19	3.80
1.00	1.35	0.99	1.01	1.28	1.49	1.34	0.86	1.27	3.22	4.46	6.14
1.00	1.42	1.59	1.17	1.35	1.72	1.26	1.03	1.02	2.10	4.50	7.48
1.00	1.06	0.95	1.02	1.01	0.82	0.84	0.85	0.71	1.62	2.25	4.32
1.00	1.37	1.02	1.41	1.15	1.15	1.07	1.06	0.87	1.92	3.09	4.62
1.00	1.20	1.20	1.25	1.09	0.84	1.23	1.29	1.19	1.65	2.35	5.16
1.00	1.14	2.09	1.91	1.12	1.27	1.35	0.91	0.87	1.65	2.55	4.63
1.00	1.11	3.53	3.46	4.59	1.71	0.78	0.55	0.54	0.87	1.00	0.85
1.00	2.58	3.06	3.71	3.67	2.43	1.16	0.87	0.62	0.85	1.08	1.05
1.00	2.41	4.84	5.49	1.01	0.69	0.49	0.52	0.46	0.82	0.77	0.90
1.00	0.73	3.08	3.55	1.02	1.26	1.19	0.95	1.17	0.76	0.93	0.78
1.00	2.67	8.14	12.69	1.50	1.58	1.72	1.23	0.77	1.14	0.99	0.89
1.00	1.47	4.14	5.40	1.00	2.56	1.04	0.67	0.97	0.59	1.12	1.16
1.00	1.01	4.73	10.62	3.00	3.62	1.61	1.06	0.86	1.67	1.53	0.80
1.00	1.50	1.51	2.48	1.68	5.97	2.33	2.12	1.91	1.30	0.97	0.94
1.00	0.59	1.01	2.22	1.71	3.33	1.65	0.78	0.44	0.90	0.67	0.73
1.00	0.69	0.77	0.93	2.45	3.86	3.10	2.10	0.55	0.68	0.64	0.81
1.00	1.05	0.63	1.92	1.32	3.50	3.11	2.14	0.99	1.13	1.13	0.95
1.00	0.75	0.53	1.37	2.74	2.81	1.66	1.96	1.18	1.03	0.91	0.85
1.00	1.29	1.05	0.73	0.72	5.63	3.50	1.94	0.94	1.33	1.69	1.88
1.00	0.95	0.91	1.01	1.07	6.95	2.54	1.47	1.46	1.39	1.13	1.26
1.00	0.69	0.83	0.48	1.35	3.17	3.24	2.86	1.56	1.16	1.15	0.92
1.00	1.05	1.41	0.94	1.01	2.96	3.27	2.91	1.82	0.94	0.71	0.64
1.00	1.19	1.76	1.72	1.53	4.79	6.12	3.42	2.25	1.58	0.88	0.82
1.00	0.95	1.47	1.91	1.21	2.59	7.23	4.63	1.79	1.29	1.22	0.85
1.00	1.17	1.47	1.31	1.17	3.12	8.91	7.43	1.21	0.97	0.78	0.73
1.00	2.67	2.41	3.75	6.18	38.80	86.92	25.58	14.81	6.73	2.72	2.42
1.00	0.83	1.67	2.70	1.55	5.06	3.10	2.36	1.72	1.60	1.41	1.77
1.00	1.07	1.46	1.64	1.51	5.17	3.02	2.18	1.55	0.82	0.89	0.85
1.00	1.19	0.82	1.40	2.33	3.40	2.66	2.23	1.38	0.96	1.12	1.15
1.00	0.71	0.87	0.79	1.71	4.34	3.10	2.81	1.19	1.11	1.11	1.16
1.00	0.61	1.25	1.13	3.09	4.11	5.69	6.03	2.70	1.60	1.49	1.14
1.00	0.90	0.73	1.35	1.65	7.93	5.70	4.59	1.92	1.59	1.13	1.03
1.00	0.81	0.50	1.29	2.04	4.52	3.38	3.51	1.38	0.87	0.69	0.71
1.00	1.11	0.88	1.79	1.47	1.87	3.00	4.03	2.31	1.01	0.68	0.70
1.00	1.06	1.56	1.05	1.55	2.18	2.84	2.98	1.77	1.28	0.90	0.60
1.00	0.81	0.94	1.07	0.82	4.51	3.60	2.62	1.97	1.09	1.69	1.68
1.00	0.84	0.92	0.96	0.89	3.44	3.24	2.44	1.51	1.59	1.44	1.75
1.00	0.71	0.90	1.06	0.84	4.89	6.63	5.37	1.09	0.95	1.55	2.21
1.00	0.77	0.78	0.79	0.78	2.71	2.03	1.92	1.73	0.86	0.91	0.70
1.00	0.60	1.08	0.74	0.76	2.23	2.84	3.12	1.41	0.68	1.26	0.85
1.00	0.57	0.71	0.98	1.00	3.16	6.78	7.18	2.38	0.81	0.94	0.87
1.00	1.11	0.68	0.97	1.02	3.35	7.25	8.18	3.13	1.96	2.28	1.58
1.00	1.19	1.27	1.23	0.99	2.19	2.07	2.75	3.39	1.57	1.46	1.29
1.00	1.10	1.36	1.53	0.77	2.48	2.31	2.12	2.28	1.44	1.07	1.10
1.00	0.73	1.81	1.41	1.10	3.02	2.87	2.47	2.20	1.10	1.06	0.95
1.00	0.69	0.87	1.18	0.66	2.04	1.94	2.29	1.48	0.90	1.28	1.34
1.00	1.12	1.37	1.33	0.93	3.55	3.13	2.27	2.28	1.56	2.34	2.14
1.00	1.05	1.22	1.20	0.81	2.30	2.59	2.61	1.92	1.81	1.51	1.84
1.00	0.98	1.25	1.20	0.78	2.56	2.83	3.57	3.47	1.88	1.71	1.33
1.00	1.09	1.13	1.49	1.60	3.28	4.30	4.01	2.47	2.36	1.99	1.76
1.00	1.26	1.30	1.49	1.13	4.49	3.94	2.82	1.55	1.81	1.55	1.47
1.00	1.20	1.25	1.11	1.64	4.33	4.53	2.90	2.13	2.00	1.69	1.64
1.00	0.60	2.30	2.18	0.71	1.33	0.97	1.48	2.03	1.04	1.54	1.50
1.00	0.82	1.05	1.76	0.82	1.64	1.20	0.75	1.16	1.24	1.88	1.99
1.00	0.70	1.97	3.37	2.22	2.90	2.14	1.76	2.14	1.58	2.79	3.62
1.00	0.76	2.11	2.01	1.30	1.92	1.75	1.96	2.73	2.49	2.83	3.13
1.00	1.17	1.22	0.73	0.85	1.11	1.42	1.90	1.66	1.63	2.45	4.04
1.00	0.86	0.69	0.62	1.02	0.97	1.11	1.13	1.24	1.80	1.95	2.69
1.00	1.07	0.65	0.97	1.18	1.37	1.39	1.92	3.04	3.44	4.53	6.13
1.00	1.07	1.00	1.11	0.93	1.28	1.52	1.45	1.50	1.82	2.66	3.84
1.00	0.71	0.85	0.89	1.15	0.97	1.07	1.75	1.69	2.53	3.89	5.84
1.00	1.16	1.49	1.40	1.41	1.40	1.86	2.26	3.27	4.06	4.43	4.70
1.00	1.24	0.92	1.00	1.39	1.49	1.12	1.56	1.97	3.24	2.68	3.19
1.00	0.99	0.71	0.77	0.98	1.26	1.02	1.32	1.95	3.30	2.95	3.26
1.00	1.11	0.72	0.86	1.08	1.10	1.12	1.55	1.39	2.07	2.24	2.71
1.00	0.95	0.80	0.95	0.88	1.10	1.24	1.68	1.70	2.36	2.56	2.44
1.00	0.97	0.66	0.71	1.08	1.07	1.25	1.70	1.70	2.04	2.09	2.56
1.00	0.87	0.56	0.70	1.38	1.55	1.78	1.94	2.95	2.68	2.58	2.67
1.00	0.93	0.69	1.10	1.32	1.08	1.30	1.96	2.36	2.11	2.71	2.88
1.00	0.91	0.66	0.78	0.83	1.24	1.82	1.31	1.55	2.52	2.13	2.08
1.00	1.14	2.92	2.75	1.11	1.31	2.18	2.69	3.37	1.87	3.57	4.01
1.00	0.32	0.68	0.77	1.38	2.09	1.79	2.39	2.29	1.49	1.70	1.50
1.00	0.81	0.55	0.73	1.17	1.70	1.72	2.50	2.40	1.09	1.42	1.03
1.00	0.91	0.64	0.81	0.83	1.18	2.11	3.33	3.68	2.71	2.40	2.27
1.00	0.44	0.75	0.80	0.72	1.51	3.27	5.54	4.37	2.42	3.97	3.26
1.00	0.44	0.49	0.61	0.98	1.05	1.39	2.18	2.42	1.91	2.33	1.65
1.00	1.19	0.92	0.68	1.15	0.97	1.95	2.81	2.99	2.35	2.25	2.07
1.00	0.74	0.62	0.86	1.12	1.08	1.65	2.98	2.99	2.26	2.21	1.80
1.00	0.79	1.02	1.09	0.66	0.88	1.31	2.61	4.20	2.29	1.76	1.39
1.00	0.44	0.52	0.53	0.34	0.62	0.58	0.70	1.08	0.89	0.79	0.78
1.00	0.88	1.05	1.09	0.72	1.07	0.88	0.99	1.54	1.90	1.83	1.71
1.00	0.70	1.20	1.15	0.68	1.03	1.22	1.98	2.16	2.14	2.01	2.40
1.00	1.00	1.94	1.62	1.08	1.61	1.72	1.56	1.62	1.89	1.95	2.35
1.00	0.74	1.10	0.43	0.77	0.85	0.75	1.06	1.96	1.64	1.54	1.37
1.00	1.23	1.02	1.31	1.32	1.61	1.58	1.36	1.48	1.65	2.33	3.10
1.00	0.98	0.53	0.76	0.98	1.88	1.37	1.23	1.27	2.17	2.92	2.20
1.00	0.90	0.57	0.68	0.84	1.48	1.65	1.94	2.58	2.23	1.87	1.52
1.00	1.05	0.86	0.65	0.99	3.08	4.27	7.53	4.93	3.75	3.02	2.48
1.00	0.61	0.48	0.65	0.67	1.29	1.30	2.36	2.09	1.14	1.51	1.74
1.00	1.12	1.32	1.11	0.75	2.01	3.27	3.95	3.86	3.00	2.75	2.39
1.00	1.02	1.05	1.02	1.12	2.20	3.42	3.81	3.53	2.89	3.30	3.37
1.00	0.70	1.01	1.14	1.26	2.21	2.87	2.98	2.32	1.71	1.52	1.25
1.00	0.81	1.13	1.18	1.11	2.23	2.24	2.37	2.35	1.95	1.63	1.66
1.00	0.83	1.05	0.99	1.04	1.64	1.92	1.75	1.98	1.68	1.31	1.32
1.00	0.89	0.81	0.87	1.47	1.96	2.08	2.45	2.45	1.78	1.60	1.54
1.00	0.90	0.88	0.94	1.59	2.42	2.80	3.01	2.47	2.20	2.19	1.55
1.00	0.67	0.90	0.99	0.99	2.40	2.72	3.89	3.16	1.73	1.59	1.24
1.00	0.90	1.08	1.28	2.58	5.03	6.26	9.22	8.54	2.74	2.47	2.88
1.00	1.13	1.01	0.99	1.98	2.94	3.51	3.97	3.54	2.93	2.40	2.19
1.00	0.83	0.70	0.73	0.74	2.27	2.55	2.80	2.85	2.01	1.52	1.40
1.00	1.27	0.63	0.88	0.84	2.71	3.75	4.97	4.91	2.77	2.51	2.29
1.00	0.78	0.66	0.77	1.06	1.74	3.75	5.36	4.63	2.13	2.44	1.93
1.00	0.92	0.97	0.90	0.83	1.90	3.75	4.97	4.33	2.32	2.21	1.88
1.00	1.14	0.96	0.98	1.49	3.18	5.41	9.55	8.05	4.48	3.02	2.10
1.00	0.88	0.91	1.27	1.40	4.48	17.43	16.41	9.50	7.37	4.45	3.97
1.00	0.80	0.58	0.44	0.57	0.90	0.94	1.50	1.39	1.27	1.82	1.75
1.00	1.34	0.99	0.80	1.26	1.12	1.56	2.06	2.44	2.49	2.63	2.31
1.00	0.92	0.93	0.74	0.74	1.17	1.63	2.56	3.20	3.57	3.77	4.18
1.00	1.32	1.06	0.81	1.72	3.62	4.13	4.96	5.56	6.84	6.18	5.31
1.00	0.56	0.76	0.62	0.91	1.91	2.16	3.74	2.60	4.84	4.79	4.52
1.00	0.70	0.81	0.89	1.57	1.94	1.82	2.46	2.36	2.74	2.62	2.23
1.00	0.92	0.98	1.03	1.38	1.82	1.96	2.06	2.17	3.19	3.36	3.42
1.00	1.15	0.93	1.17	1.78	2.18	1.96	2.01	3.03	3.35	2.53	2.59
1.00	0.93	0.61	0.79	0.91	1.06	1.38	1.78	1.89	1.75	1.49	1.45
1.00	0.73	0.70	0.73	1.04	1.31	1.24	1.61	1.87	1.69	1.78	2.01
1.00	0.95	0.71	0.97	1.21	2.09	1.98	2.33	2.08	1.78	2.10	2.37
1.00	0.95	0.81	0.97	1.40	1.31	1.36	2.00	1.97	2.09	2.09	2.22
1.00	0.89	0.78	1.02	1.00	1.59	1.75	2.33	2.43	2.26	2.38	2.45
1.00	0.76	0.76	0.89	1.10	1.77	2.36	3.43	3.00	4.49	3.69	3.21
1.00	1.21	1.16	1.32	1.46	2.40	3.43	4.29	3.97	4.93	3.55	2.96
1.00	1.16	2.28	2.27	1.09	2.81	3.16	3.66	4.36	2.80	3.12	2.54
1.00	0.96	1.27	0.97	1.19	1.46	1.76	1.94	2.40	2.31	1.81	1.53
1.00	0.85	1.12	1.10	0.75	1.12	1.55	1.65	2.29	1.61	1.48	1.49
1.00	0.74	1.24	1.15	0.72	1.50	1.65	1.63	2.23	1.88	1.65	1.65
1.00	0.73	0.95	0.93	0.89	1.39	1.69	1.87	1.89	2.02	2.04	1.87
1.00	0.72	1.03	0.65	0.63	1.16	1.74	1.88	1.99	2.03	2.21	2.35
1.00	0.56	1.07	0.57	0.80	1.45	2.68	2.70	2.86	3.32	2.95	2.61
1.00	0.76	1.23	1.07	0.46	2.04	3.32	3.73	3.80	3.54	4.91	4.13
1.00	1.08	1.06	0.94	1.01	1.32	1.39	2.37	2.44	1.90	1.66	1.45
1.00	0.78	0.88	0.75	0.68	1.67	1.98	2.70	3.39	1.98	2.24	1.82
1.00	0.95	1.20	1.32	1.14	2.29	2.21	2.80	2.25	3.11	2.44	2.54
1.00	1.03	1.12	1.00	1.05	2.11	3.21	3.58	3.70	3.19	3.36	3.04
1.00	1.07	0.76	1.02	0.90	3.57	4.44	6.80	7.00	7.33	5.95	5.13
1.00	0.90	1.15	1.53	1.22	5.33	7.75	11.72	10.86	11.06	11.67	11.02
1.00	0.99	1.32	1.65	0.75	5.76	8.51	10.02	9.28	12.34	9.12	8.81
1.00	1.14	1.25	1.24	1.65	7.22	17.97	21.16	11.20	15.21	13.75	6.39
1.00	0.75	1.19	1.23	0.77	0.84	1.06	1.45	1.80	1.03	1.20	1.06
1.00	1.16	1.34	1.55	1.08	1.91	1.98	2.37	2.76	2.00	1.70	1.15
1.00	1.26	2.57	1.52	0.98	1.18	1.70	2.56	3.24	1.38	1.43	1.07
1.00	1.07	1.85	1.55	1.07	1.42	1.60	2.29	2.54	1.14	1.16	0.95
1.00	0.85	1.22	1.16	0.96	1.16	1.38	1.92	1.78	0.90	0.90	0.71
1.00	1.03	1.22	1.20	1.00	1.68	1.66	1.87	2.38	1.16	0.81	1.05
1.00	0.98	1.35	1.78	1.19	1.35	1.36	1.72	2.57	1.34	1.16	1.10
1.00	1.19	1.61	1.62	2.04	2.04	2.15	3.43	4.16	2.62	2.37	1.81
1.00	1.21	2.85	2.15	2.04	2.95	3.00	4.49	4.91	2.17	1.90	1.36
1.00	0.96	1.17	0.87	2.62	2.28	2.20	1.24	2.03	1.22	1.02	0.88
1.00	0.96	1.49	1.48	3.59	4.14	3.03	3.57	2.06	2.20	2.28	1.86
1.00	1.08	0.92	1.53	1.91	3.25	2.36	1.77	1.72	2.16	1.77	1.84
1.00	0.39	1.40	1.74	2.07	3.15	3.80	4.34	2.97	2.09	1.71	1.44
1.00	0.92	1.02	1.64	2.36	3.02	2.37	3.05	1.75	1.69	1.52	1.61
1.00	0.94	0.94	1.66	2.70	4.38	3.06	2.75	2.14	1.48	1.73	1.34
1.00	1.05	0.76	1.53	3.22	4.43	3.60	2.63	2.22	1.80	1.39	1.19
1.00	1.06	1.23	2.36	2.41	3.27	3.75	2.40	1.65	1.61	2.09	1.85
1.00	1.18	5.53	11.90	3.86	28.22	20.07	16.54	9.58	3.30	5.88	8.33
1.00	0.76	1.42	3.61	5.75	3.23	2.61	7.69	7.13	5.10	4.12	4.96
1.00	1.40	1.68	2.09	3.18	1.42	1.38	1.64	1.74	2.33	2.19	2.67
1.00	0.78	0.66	4.69	5.25	1.16	1.78	2.30	1.65	1.78	1.45	1.59
1.00	1.05	0.85	1.20	2.05	2.33	1.94	2.66	1.66	0.71	0.65	0.67
1.00	1.01	0.78	1.17	1.87	1.60	1.32	1.85	1.54	0.96	0.66	0.68
1.00	1.03	1.89	1.46	1.08	1.03	1.02	1.95	1.80	0.65	0.76	0.61
1.00	1.50	2.36	3.63	3.30	1.34	2.28	1.82	1.76	1.02	0.99	1.04
1.00	0.68	1.04	1.17	0.84	1.15	0.87	0.76	0.59	0.61	0.63	0.83
1.00	0.90	1.19	2.59	1.06	1.24	1.76	1.51	0.82	0.87	0.92	0.72
1.00	0.92	1.06	6.60	5.84	5.39	2.09	1.47	1.16	0.91	0.83	0.82
1.00	0.72	0.64	3.46	1.87	1.53	1.57	1.30	0.81	0.61	0.72	0.76
1.00	0.81	1.14	4.47	5.65	4.28	2.59	2.26	1.02	1.11	1.17	1.12
1.00	2.27	2.64	6.39	8.01	3.34	1.88	1.85	1.16	1.10	0.98	0.87
1.00	1.14	2.14	2.61	2.54	1.73	2.20	2.07	1.18	0.94	0.94	0.92
1.00	1.15	3.36	4.32	3.21	4.62	3.89	2.42	1.40	1.07	0.99	1.20
1.00	1.07	1.78	2.74	2.78	3.09	3.84	3.54	1.80	1.08	1.04	0.83
1.00	1.37	2.05	3.88	3.43	3.45	4.04	4.04	2.20	1.45	0.84	0.92
1.00	1.65	3.02	5.08	11.48	12.58	10.44	10.53	3.42	1.15	0.86	0.86
1.00	0.97	1.63	3.65	5.11	0.92	1.84	2.66	2.14	1.24	1.23	1.05
1.00	1.85	2.08	3.98	3.39	1.29	2.27	2.73	2.16	1.20	0.97	0.90
1.00	1.33	2.86	3.07	1.85	1.95	1.88	1.75	1.44	1.16	1.35	1.48
1.00	1.30	2.78	5.20	2.95	2.31	3.08	2.77	1.87	1.77	1.66	1.72
1.00	2.41	3.57	7.18	4.86	3.10	3.58	3.93	2.28	2.19	2.16	2.23
1.00	2.09	3.37	5.52	4.89	3.05	3.27	4.29	2.72	2.15	2.05	2.25
1.00	1.52	4.39	7.03	5.45	2.93	3.91	4.44	2.40	1.97	2.21	1.79
1.00	2.25	4.67	7.94	5.94	3.76	4.46	4.60	2.43	1.86	1.57	1.26
